# Workflow demonstration with a recommender engine on a sampled dataset from Transactions.csv using ALS Model
### This is the notebook for deployment

In [51]:
# LIBRARIES
#!pip install pyspark
#!pip install --upgrade google-cloud-bigquery[pandas]
#!pip install pyspark[sql] #PARSEO RAPIDO DE PANDAS A SPARK RDDSQL

ERROR: Could not find a version that satisfies the requirement dsx_ml (from versions: none)
ERROR: No matching distribution found for dsx_ml


### Importing the libraries and starting the Spark Session

In [2]:
#!export ARROW_PRE_0_15_IPC_FORMAT=1
!echo $ARROW_PRE_0_15_IPC_FORMAT

1


In [3]:
!echo $GOOGLE_APPLICATION_CREDENTIALS
!echo $JAVA_HOME
!echo $SPARK_HOME
!java -version

/home/antonio/Desktop/github/RMF2Coppel/BQkeys/antoniobq_key.json
/usr/lib/jvm/java-8-openjdk-amd64
/opt/spark
openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1ubuntu1-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)


In [4]:
import pyspark.sql.functions as sql_func
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.context import SparkContext 
from pyspark.sql import SparkSession
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd

In [5]:
sc = SparkContext.getOrCreate()

In [6]:
spark = SparkSession(sc)
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

### Add asset from remote connection 

In [7]:
import  requests, os, io #dsx_core_utils
#final_stat = None
final_stat = pd.read_csv('Workflow_aggregado.csv')
final_stat.columns = ['ID_CTE', 'ID_CLAS1', 'FREQUENCY']
final_stat.head(10)

,ID_CTE,ID_CLAS1,FREQUENCY
0,31936884,541060,10
1,5916508,314063,6
2,13173248,224065,9
3,4532176,314063,7
4,18164173,314063,13
5,25655734,594244,6
6,32812328,104014,6
7,3000656,106061,7
8,36507674,420083,6
9,27660076,477092,10


In [8]:
# Get data from GCP- BQ
#from google.cloud import bigquery
#def get_data_BQ(sql):
#    client = bigquery.Client()
#    df = client.query(sql).to_dataframe()
#    return(df)
#sql =  '''SELECT ID_CTE as ID_CTE, ID_FAM as ID_CLAS1, FREQUENCY as FREQUENCY
#FROM `rmf2gcp.RawData.Workflow_aggregado`
#limit 3105886#310 588 606 ''' # corre en mi local y pesa 56MB %1 del total de la muestra

In [9]:
#data = get_data_BQ(sql)

In [11]:
print(final_stat.dtypes)
print(final_stat.shape)

ID_CTE       int64
ID_CLAS1     int64
FREQUENCY    int64
dtype: object
(3105886, 3)


In [12]:
final_stat = spark.createDataFrame(final_stat)
final_stat.show(5)

+--------+--------+---------+
|  ID_CTE|ID_CLAS1|FREQUENCY|
+--------+--------+---------+
|31936884|  541060|       10|
| 5916508|  314063|        6|
|13173248|  224065|        9|
| 4532176|  314063|        7|
|18164173|  314063|       13|
+--------+--------+---------+
only showing top 5 rows



In [13]:
#from pyspark.sql.types import IntegerType
#for i in final_stat.columns:
 #   final_stat = final_stat.withColumn(i, final_stat[i].cast(IntegerType()))
final_stat.count()

3105886

### Preparing data for the model

In [15]:
ratings = (final_stat
    .select(
        'ID_CTE',
        'ID_CLAS1',
        'FREQUENCY',
    )
).cache()

In [17]:
#ratings.limit(310588)

In [33]:
ratings.count()

3105886

### Spliting the data set to test and train for measuring the performance of the ALS Model

In [18]:
(training, test) = ratings.randomSplit([0.8, 0.2])

### Build the recommendation model using ALS on the training data


In [19]:
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=2, regParam=0.01, 
          userCol="ID_CTE", itemCol="ID_CLAS1", ratingCol="FREQUENCY",
          coldStartStrategy="drop",
          implicitPrefs=True)

In [21]:
model = als.fit(ratings)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="FREQUENCY",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 2.011249846550589


In [23]:
predictions.show(5)

+--------+--------+---------+--------------+
|  ID_CTE|ID_CLAS1|FREQUENCY|    prediction|
+--------+--------+---------+--------------+
|26611533|  212010|        1|-1.33456455E-8|
|24858262|  212010|        1|   4.194458E-9|
|39868047|  212010|        1|   4.194458E-9|
|32473582|  212010|        1|   4.194458E-9|
|14107606|  212010|        1|   3.229041E-6|
+--------+--------+---------+--------------+
only showing top 5 rows



## Parameters of ALS Model in PySpark realization are following:

##### NumBlocks is the number of blocks the users and items will be partitioned into in order to parallelize computation.
##### rank is the number of latent factors in the model.
##### maxIter is the maximum number of iterations to run.
##### regParam specifies the regularization parameter in ALS.
##### implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data (defaults to false which means using explicit feedback).
##### alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations (defaults to 1.0)

###  Generate top 10 Item recommendations for each user



In [24]:
userRecs = model.recommendForAllUsers(10)
print(userRecs.count())
userRecs.show(5)


2649183
+------+--------------------+
|ID_CTE|     recommendations|
+------+--------------------+
| 19553|[[106059, 0.01266...|
| 19984|[[104014, 0.00532...|
| 29719|[[314129, 0.01303...|
| 37307|[[860048, 0.27645...|
| 38723|[[102011, 4.50272...|
+------+--------------------+
only showing top 5 rows



In [34]:
help(model.recommendForAllUsers)

Help on method recommendForAllUsers in module pyspark.ml.recommendation:

recommendForAllUsers(numItems) method of pyspark.ml.recommendation.ALSModel instance
    Returns top `numItems` items recommended for each user, for all users.
    
    :param numItems: max number of recommendations for each user
    :return: a DataFrame of (userCol, recommendations), where recommendations are
             stored as an array of (itemCol, rating) Rows.
    
    .. versionadded:: 2.2.0



In [25]:
userRecs[['recommendations']].show()

+--------------------+
|     recommendations|
+--------------------+
|[[106059, 0.01266...|
|[[104014, 0.00532...|
|[[314129, 0.01303...|
|[[860048, 0.27645...|
|[[102011, 4.50272...|
|[[102089, 0.05551...|
|[[314156, 0.56780...|
|[[102089, 0.03627...|
|[[314063, 0.04459...|
|[[319064, 0.03723...|
|[[380284, 0.01422...|
|[[701305, 7.63655...|
|[[106003, 0.60024...|
|[[224041, 0.01021...|
|[[106055, 0.80115...|
|[[104014, 0.20452...|
|[[319064, 0.06844...|
|[[224065, 0.32506...|
|[[106061, 0.01819...|
|[[121001, 0.16221...|
+--------------------+
only showing top 20 rows



In [28]:
print(help(userRecs.take))
userRecs.take(2)

Help on method take in module pyspark.sql.dataframe:

take(num) method of pyspark.sql.dataframe.DataFrame instance
    Returns the first ``num`` rows as a :class:`list` of :class:`Row`.
    
    >>> df.take(2)
    [Row(age=2, name='Alice'), Row(age=5, name='Bob')]
    
    .. versionadded:: 1.3

None


[Row(ID_CTE=19553, recommendations=[Row(ID_CLAS1=106059, rating=0.012666523456573486), Row(ID_CLAS1=104015, rating=0.012505062855780125), Row(ID_CLAS1=224017, rating=0.012294436804950237), Row(ID_CLAS1=319064, rating=0.01215100847184658), Row(ID_CLAS1=106061, rating=0.011899995617568493), Row(ID_CLAS1=224025, rating=0.011808079667389393), Row(ID_CLAS1=318009, rating=0.010645071975886822), Row(ID_CLAS1=423132, rating=0.008973229676485062), Row(ID_CLAS1=105074, rating=0.008193274959921837), Row(ID_CLAS1=101004, rating=0.007982431910932064)]),
 Row(ID_CTE=19984, recommendations=[Row(ID_CLAS1=104014, rating=0.005321032367646694), Row(ID_CLAS1=104015, rating=0.004974351730197668), Row(ID_CLAS1=224025, rating=0.004570256918668747), Row(ID_CLAS1=291059, rating=0.004364300053566694), Row(ID_CLAS1=224017, rating=0.004133342765271664), Row(ID_CLAS1=869212, rating=0.0032216128893196583), Row(ID_CLAS1=224065, rating=0.003081432543694973), Row(ID_CLAS1=314156, rating=0.0025232555344700813), Row(ID_

### Display the recommendations and get them in the correct format

In [29]:
from pyspark.sql.functions import explode
userRecs1=userRecs.withColumn("recommendations", explode(userRecs.recommendations))
userRecs1.show()

+------+--------------------+
|ID_CTE|     recommendations|
+------+--------------------+
| 19553|[106059, 0.012666...|
| 19553|[104015, 0.012505...|
| 19553|[224017, 0.012294...|
| 19553|[319064, 0.012151...|
| 19553|[106061, 0.011899...|
| 19553|[224025, 0.01180808]|
| 19553|[318009, 0.010645...|
| 19553|[423132, 0.00897323]|
| 19553|[105074, 0.008193...|
| 19553|[101004, 0.007982...|
| 19984|[104014, 0.005321...|
| 19984|[104015, 0.004974...|
| 19984|[224025, 0.004570...|
| 19984| [291059, 0.0043643]|
| 19984|[224017, 0.004133...|
| 19984|[869212, 0.003221...|
| 19984|[224065, 0.003081...|
| 19984|[314156, 0.002523...|
| 19984|[106001, 0.002389...|
| 19984|[787254, 0.002262...|
+------+--------------------+
only showing top 20 rows



In [38]:
#ratings.unpersist(True)

DataFrame[ID_CTE: bigint, ID_CLAS1: bigint, FREQUENCY: bigint]

####  Breaking down reach recommendation to separate columns

In [30]:
userRecs1= userRecs1.select('ID_CTE', 'recommendations.*')       

### Display the results

In [31]:
userRecs1.show() 

+------+--------+------------+
|ID_CTE|ID_CLAS1|      rating|
+------+--------+------------+
| 19553|  106059| 0.012666523|
| 19553|  104015| 0.012505063|
| 19553|  224017| 0.012294437|
| 19553|  319064|0.0121510085|
| 19553|  106061| 0.011899996|
| 19553|  224025|  0.01180808|
| 19553|  318009| 0.010645072|
| 19553|  423132|  0.00897323|
| 19553|  105074| 0.008193275|
| 19553|  101004| 0.007982432|
| 19984|  104014|0.0053210324|
| 19984|  104015|0.0049743517|
| 19984|  224025| 0.004570257|
| 19984|  291059|   0.0043643|
| 19984|  224017|0.0041333428|
| 19984|  869212| 0.003221613|
| 19984|  224065|0.0030814325|
| 19984|  314156|0.0025232555|
| 19984|  106001| 0.002389164|
| 19984|  787254| 0.002262605|
+------+--------+------------+
only showing top 20 rows



In [32]:
userRecs1.count()

26491830

### Writing the Output back to the Remote Datasource

In [35]:
final_stat = userRecs1.toPandas()

In [41]:
userRecs1.unpersist(True)

DataFrame[ID_CTE: int, ID_CLAS1: int, rating: float]

In [42]:
final_stat.to_csv('test_local_10_junio_2020.csv')

In [43]:
!zip test_local_10_junio_2020.csv.zip test_local_10_junio_2020.csv

  adding: test_local_10_junio_2020.csv (deflated 70%)


In [44]:
# borramos los resultados de momento porque son bastente grandes aun compresos como para guardarlos en el repo 
!rm test_local_10_junio_2020.csv
!rm test_local_10_junio_2020.csv.zip

In [45]:
from pyspark.ml import Pipeline

In [46]:
pipeline = Pipeline(stages=[model])

In [47]:
model_alsWML = pipeline.fit(ratings)

In [63]:
model_alsWML.save('/home/antonio/Desktop/github/RMF2Coppel/CodeAntonio (local)/Experiment_local/')

In [65]:
# :D END